In [1]:
'''
S&P500의 종목들의 2010년 이후 가격, 거래량 정보 추출
'''
import sqlite3
import pandas as pd


conn = sqlite3.connect('Database/S&P500.db')
query = "SELECT * FROM stock"
stock = pd.read_sql_query(query, conn)
# ticker_by_sector = stock.groupby(by='gics_sector')['symbol'].apply(list).to_dict()
tickers = list(stock['symbol'].unique())

conn.close()

In [49]:
import yfinance as yf


company = yf.Ticker('AAPL')

# 손익계산서 (Income Statement)
income_statement = company.quarterly_financials
# 재무상태표 (Balance Sheet)
balance_sheet = company.quarterly_balance_sheet
# 현금흐름표 (Cash Flow Statement)
cash_flow = company.quarterly_cashflow

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(list(income_statement.index))
print(list(balance_sheet.index))
print(list(cash_flow.index))

['Tax Effect Of Unusual Items', 'Tax Rate For Calcs', 'Normalized EBITDA', 'Total Unusual Items', 'Total Unusual Items Excluding Goodwill', 'Net Income From Continuing Operation Net Minority Interest', 'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA', 'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income', 'Normalized Income', 'Net Income From Continuing And Discontinued Operation', 'Total Expenses', 'Total Operating Income As Reported', 'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders', 'Net Income Common Stockholders', 'Net Income', 'Minority Interests', 'Net Income Including Noncontrolling Interests', 'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income', 'Other Income Expense', 'Other Non Operating Income Expenses', 'Special Income Charges', 'Gain On Sale Of Business', 'Other Special Charges', 'Impairment Of Capital Assets', 'Restructuring And Mergern Acquisition', 'Gain 

In [2]:
import yfinance as yf
import pandas as pd

def get_quarterly_financials(ticker):
    # 종목 정보 가져오기
    stock = yf.Ticker(ticker)

    # 분기별 재무제표 가져오기
    income_statement = stock.quarterly_financials.T
    balance_sheet = stock.quarterly_balance_sheet.T
    cash_flow = stock.quarterly_cashflow.T

    return income_statement, balance_sheet, cash_flow

def calculate_financial_metrics_from_yfinance(ticker):
    # yfinance 데이터 가져오기
    income_statement, balance_sheet, cash_flow = get_quarterly_financials(ticker)

    # 필요한 데이터 추출 및 계산
    metrics = {}

    # (1) 부채 관련 지표
    metrics['Debt_to_Equity_Ratio'] = balance_sheet['Total Debt'] / balance_sheet['Stockholders Equity']
    metrics['Interest_Coverage_Ratio'] = income_statement['EBIT'] / income_statement['Interest Expense']
    metrics['Net_Debt_to_EBITDA'] = (balance_sheet['Total Debt'] - balance_sheet['Cash And Cash Equivalents']) / income_statement['EBITDA']

    # (2) 수익성 관련 지표
    # metrics['Net_Profit_Margin'] = (income_statement['Net Income'] / income_statement['Total Revenue']) * 100
    # metrics['Operating_Margin'] = (income_statement['Operating Income'] / income_statement['Total Revenue']) * 100

    # (3) 현금 흐름 관련 지표
    metrics['Free_Cash_Flow'] = income_statement['Net Income'] + cash_flow['Depreciation Amortization Depletion'] - cash_flow['Capital Expenditure']

    # (4) 성장성 관련 지표 (전 분기 대비 계산 필요)
    # metrics['EPS_Growth_Rate'] = ((income_statement['Diluted EPS'] - income_statement['Diluted EPS'].shift(1)) / income_statement['Diluted EPS'].shift(1)) * 100
    # metrics['Revenue_Growth_Rate'] = ((income_statement['Total Revenue'] - income_statement['Total Revenue'].shift(1)) / income_statement['Total Revenue'].shift(1)) * 100

    # (5) 유동성 관련 지표
    # metrics['Current_Ratio'] = balance_sheet['Current Assets'] / balance_sheet['Current Liabilities']
    # metrics['Quick_Ratio'] = (balance_sheet['Current Assets'] - balance_sheet['Inventory']) / balance_sheet['Current Liabilities']

    # 결과를 DataFrame으로 변환
    metrics_df = pd.DataFrame(metrics)
    return metrics_df.sort_index(ascending=False)

In [57]:
stock = yf.Ticker(ticker)

# 분기별 재무제표 가져오기
income_statement = stock.quarterly_financials.T
balance_sheet = stock.quarterly_balance_sheet.T
cash_flow = stock.quarterly_cashflow.T

display(income_statement)

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,Interest Expense,Interest Income,Normalized Income,Net Income From Continuing And Discontinued Operation,Total Expenses,Total Operating Income As Reported,Diluted Average Shares,Basic Average Shares,Diluted EPS,Basic EPS,Diluted NI Availto Com Stockholders,Net Income Common Stockholders,Net Income,Minority Interests,Net Income Including Noncontrolling Interests,Net Income Continuous Operations,Tax Provision,Pretax Income,Other Income Expense,Other Non Operating Income Expenses,Special Income Charges,Gain On Sale Of Business,Other Special Charges,Impairment Of Capital Assets,Restructuring And Mergern Acquisition,Gain On Sale Of Security,Net Non Operating Interest Income Expense,Interest Expense Non Operating,Interest Income Non Operating,Operating Income,Operating Expense,Depreciation Amortization Depletion Income Statement,Depreciation And Amortization In Income Statement,Amortization,Amortization Of Intangibles Income Statement,Depreciation Income Statement,Research And Development,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Other Gand A,Salaries And Wages,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
2024-09-30,2170000.0,0.217,1136000000.0,10000000.0,10000000.0,684000000.0,211000000.0,388000000.0,1146000000.0,935000000.0,6000000.0,61000000.0,67000000.0,676170000.0,684000000.0,3200000000.0,762000000.0,135732000.0,129758000.0,5.04,5.28,684000000.0,684000000.0,684000000.0,0.0,684000000.0,684000000.0,190000000.0,874000000.0,8000000.0,-2000000.0,-96000000.0,2000000.0,59000000.0,33000000.0,6000000.0,106000000.0,6000000.0,61000000.0,67000000.0,860000000.0,2812000000.0,211000000.0,211000000.0,14000000.0,14000000.0,197000000.0,320000000.0,2281000000.0,2052000000.0,229000000.0,36000000.0,193000000.0,3672000000.0,388000000.0,4060000000.0,4060000000.0
2024-06-30,-3262000.0,0.233,768000000.0,-14000000.0,-14000000.0,386000000.0,205000000.0,362000000.0,754000000.0,549000000.0,6000000.0,61000000.0,67000000.0,396738000.0,386000000.0,3068000000.0,451000000.0,137832000.0,131948000.0,2.8,2.92,386000000.0,386000000.0,386000000.0,11000000.0,375000000.0,375000000.0,113000000.0,488000000.0,-8000000.0,6000000.0,-39000000.0,0.0,21000000.0,NaN,18000000.0,25000000.0,6000000.0,61000000.0,67000000.0,490000000.0,2706000000.0,205000000.0,205000000.0,15000000.0,15000000.0,190000000.0,331000000.0,2170000000.0,1990000000.0,180000000.0,36000000.0,144000000.0,3196000000.0,362000000.0,3558000000.0,3558000000.0
2024-03-31,-12896000.0,0.124,221000000.0,-104000000.0,-104000000.0,-135000000.0,210000000.0,358000000.0,117000000.0,-93000000.0,-11000000.0,62000000.0,51000000.0,-43896000.0,-135000000.0,2931000000.0,-110000000.0,135501000.0,135501000.0,-0.99,-0.99,-135000000.0,-135000000.0,-135000000.0,1000000.0,-136000000.0,-136000000.0,-19000000.0,-155000000.0,-102000000.0,2000000.0,-65000000.0,3000000.0,20000000.0,NaN,48000000.0,-39000000.0,-11000000.0,62000000.0,51000000.0,-42000000.0,2573000000.0,210000000.0,210000000.0,15000000.0,15000000.0,195000000.0,341000000.0,2022000000.0,1836000000.0,186000000.0,38000000.0,148000000.0,2531000000.0,358000000.0,2889000000.0,2889000000.0
2023-12-31,-6549707.602339,0.204678,472000000.0,-32000000.0,-32000000.0,132000000.0,208000000.0,340000000.0,440000000.0,232000000.0,-16000000.0,61000000.0,45000000.0,157450292.397661,132000000.0,2667000000.0,104000000.0,144470000.0,138184000.0,0.92,0.96,132000000.0,132000000.0,132000000.0,-4000000.0,136000000.0,136000000.0,35000000.0,171000000.0,-33000000.0,-1000000.0,-115000000.0,1000000.0,2000000.0,114000000.0,NaN,83000000.0,-16000000.0,61000000.0,45000000.0,220000000.0,2327000000.0,208000000.0,208000000.0,15000000.0,15000000.0,193000000.0,357000000.0,1762000000.

In [3]:
def calculate_grades(row):
    # Debt-to-Equity Ratio 등급 계산
    if 0.0 <= row['Debt_to_Equity_Ratio'] < 0.5:
        debt_to_equity_grade = 1
    elif 0.5 <= row['Debt_to_Equity_Ratio'] < 1.0:
        debt_to_equity_grade = 2
    elif 1.0 <= row['Debt_to_Equity_Ratio'] < 1.5:
        debt_to_equity_grade = 3
    elif 1.5 <= row['Debt_to_Equity_Ratio'] < 2.0:
        debt_to_equity_grade = 4
    else:
        debt_to_equity_grade = 5

    # Interest Coverage Ratio 등급 계산
    if row['Interest_Coverage_Ratio'] >= 5.0:
        interest_coverage_grade = 1
    elif 3.0 <= row['Interest_Coverage_Ratio'] < 5.0:
        interest_coverage_grade = 2
    elif 1.5 <= row['Interest_Coverage_Ratio'] < 3.0:
        interest_coverage_grade = 3
    elif 1.0 <= row['Interest_Coverage_Ratio'] < 1.5:
        interest_coverage_grade = 4
    else:
        interest_coverage_grade = 5

    # Net Debt-to-EBITDA 등급 계산
    if 0.0 <= row['Net_Debt_to_EBITDA'] < 1.0:
        net_debt_to_ebitda_grade = 1
    elif 1.0 <= row['Net_Debt_to_EBITDA'] < 2.0:
        net_debt_to_ebitda_grade = 2
    elif 2.0 <= row['Net_Debt_to_EBITDA'] < 3.0:
        net_debt_to_ebitda_grade = 3
    elif 3.0 <= row['Net_Debt_to_EBITDA'] < 4.0:
        net_debt_to_ebitda_grade = 4
    else:
        net_debt_to_ebitda_grade = 5

    # Free Cash Flow 등급 계산
    if row['Free_Cash_Flow'] > 0 and row['Free_Cash_Flow'] > 1000000:
        free_cash_flow_grade = 1
    elif row['Free_Cash_Flow'] > 0:
        free_cash_flow_grade = 2
    elif row['Free_Cash_Flow'] == 0:
        free_cash_flow_grade = 3
    elif row['Free_Cash_Flow'] < 0 and row['Free_Cash_Flow'] > -1000000:
        free_cash_flow_grade = 4
    else:
        free_cash_flow_grade = 5

    return debt_to_equity_grade, interest_coverage_grade, net_debt_to_ebitda_grade, free_cash_flow_grade

In [4]:
# 예제 실행 (AAPL 종목으로 테스트)
lst = []

for ticker in tickers:
    try:
        metrics_df = calculate_financial_metrics_from_yfinance(ticker)
        metrics_df[['Debt_to_Equity_Grade', 'Interest_Coverage_Grade', 'Net_Debt_to_EBITDA_Grade', 'Free_Cash_Flow_Grade']] = metrics_df.apply(calculate_grades, axis=1, result_type='expand')
        metrics_df = metrics_df[['Debt_to_Equity_Grade', 'Interest_Coverage_Grade', 'Net_Debt_to_EBITDA_Grade', 'Free_Cash_Flow_Grade']].iloc[0]
        if (all(metrics_df.values >= 2)):
            lst.append(ticker)
    except:
        continue
    
print(lst)

['AEE', 'BA', 'STZ', 'INTC', 'NRG', 'SMCI', 'TTWO', 'TRMB', 'WBA', 'WTW']
